In [75]:
import pandas as pd
import numpy

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

#Import data
all_colors = pd.read_csv("data/all-colors.csv")
chosen_colors = pd.read_csv("data/chosen-colors.csv")

#Remove chosen colors
condition = all_colors["Shirt Color Name"].isin(chosen_colors["Shirt Color"])
remaining_colors = all_colors.drop(all_colors[condition].index, inplace=False)

#Confirm removal
# print(len(remaining_colors))
# print(len(all_colors))
# print(len(chosen_colors))

In [80]:
import colormath.color_objects as co
import colormath.color_diff as cdiff

def compute_average_distance(row):
    #color_name = row["Shirt Color"]
    color_Lab = None
    try:
        color_Lab = co.LabColor(*map(float, row["L*a*b Value"].split(", ")))
    except ValueError:
            print("there was an error")
            print("The color is " + row["Shirt Color"])
    distances = []

    #Compute all distances
    for index, row_remaining in remaining_colors.iterrows():
        other_color = co.LabColor(*map(float, row_remaining["L*a*b Value"].split(", ")))

        #Human perceived distance from target to remaining colors, based off perception 1,1
        try:
            distance = cdiff.delta_e_cmc(color_Lab, other_color, pl=1, pc=1)
        except ValueError:
            print("there was an error")
            print("The color is" + row_remaining["Shirt Color"])

        distances += [distance]

    average_distance = sum(distances)/len(remaining_colors)

    row["Distance"] = average_distance
    return row

remaining_colors = remaining_colors.apply(compute_average_distance, axis=1)

In [83]:
sorted_remaining = remaining_colors.sort_values(by = "Distance", ascending=False)
sorted_remaining.iloc[:].head(10)

,Shirt Color Name,PMS Color Code,L*a*b Value,Hex,Distance
51,Vintage Black,419 C,"13.35, -1.03, 0.45",212322,97.929101
48,Team Purple,2695C,"14.26, 17.59, -24.18",2E1A47,90.085017
7,Black,Black C,"17.06, 1.29, 2.54",2D2926,85.494305
31,Navy,533C,"16.99, 2.57, -18.63",202A44,85.408818
18,Forest,5535 C,"17.54, -12.05, 1.20",183029,79.977051
77,Heather Midnight Navy,2380 CP,"24.29, 1.30, -11.02",343A4A,63.893376
8,Brown,440CP,"25.05, 3.33, 6.64",423A31,60.741023
107,Solid White Blend,WhiteC,"100, 0, 0",FFFFFF,58.862833
63,Heather Cardinal,7421 C,"24.16, 35.71, 6.28",651C32,56.781678
56,Black Heather,BLACK 7 CP,"27.28, 2.57, 5.49",454038,56.202490
